In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
from skinet.utils.athena_tools import *
warnings.filterwarnings("ignore")

In [2]:
# matplotlib 한글 나오게 하기

import matplotlib as mpl
import matplotlib.pylab as plt2 

plt2.rcParams["font.family"] = "NanumGothincCoding"
plt2.rcParams["font.size"] = 12
plt2.rcParams["figure.figsize"] = (14, 4) 
mpl.rcParams['axes.unicode_minus'] = False 

In [3]:
ym = '202010'                          # 전월 (몇월의 마감자료로 작업하고 있는지)
ym_minus_6month = '202004'             # 6개월 전 (해약문의 등의 기준)
first_bill_month = '202009'            # 최초 청구월 기준
final_call_boundary = 20200901         # 최종 통화일 기준 (이 이후 통화건이 있으면 제외)

## 1. 월 마감자료 만들기

In [4]:
conn = athena_connection()
drop_table_in_athena(conn, 'default.monthly_deadline_{}'.format(ym))
magam = load_from_athena(conn, '''
SELECT a.kcalss
	 , d.kunnr
	 , d.ctnum
     , a.conn_dept
     , a.conn_emp
     , a.cost_dept
     , a.cost_emp
     , a.ctnum as ctnum_hash
     , a.kunnr as kunnr_hash
     , a.cstat
     , a.gad_name
     , a.gad_start
     , a.conn_start
     , a.conn_end
     , CAST(a.monexp_amt AS DOUBLE)*100 AS monexp_amt
     , a.conn_type
     , a.ctype_l
     , a.ctype_m
     , a.ctype_s
     , a.intro_asp
     , a.itype_l
     , a.itype_m
     , a.itype_s
     , a.itype_sdesc
     , c.name1
     , a.gad_add01
     , a.post_add01
     , a.first_bildate
     , c.telf2
     , a.svc_str
     , a.pay_type
     , a.depo_type
     , a.line_type
     , a.pck_str
     , c.stcd2
     , c.j_1kftbus AS uptae
     , c.j_1kftind AS jongmok
     , a.gros_code
     , a.gross_type
     , a.kvgr4
     , a.gad_pstlz
     , a.gad_add02
     , a.post_add02
     , b.sms_yn
     , b.email_yn
     , b.tm_yn
     , b.dm_yn
     , b.sk1_yn
     , b.sk2_yn
     , b.adt1_yn
     , b.aedat
     , b.aenam
     , e.ctrt_id AS nsok
	 , ROW_NUMBER() OVER (PARTITION BY d.kunnr ORDER BY a.conn_type ASC, d.ctnum DESC) AS kunnr_rownum
FROM sap.zsdt4013      AS a 
LEFT JOIN (SELECT ctnum, kunnr, sms_yn, email_yn, tm_yn, dm_yn, sk1_yn, sk2_yn, adt1_yn, aedat, aenam FROM sap.zsdt3748 WHERE usest = 'X') AS b
      ON a.ctnum = b.ctnum
LEFT JOIN (SELECT kunnr, stcd2, stcd1, telf2, name1, j_1kftbus, j_1kftind FROM sap.kna1 WHERE mandt = '100') AS c
      ON a.kunnr = c.kunnr
LEFT JOIN (select distinct ctnum_hash, ctnum, kunnr_hash, kunnr from adt_work.hash_info) d
       ON a.ctnum = d.ctnum_hash
LEFT JOIN (select distinct ctrt_id, ctnum from telepro.tb_nsok_mapping) e --n사 고객 추출
       ON a.ctnum = e.ctnum
  WHERE a.mandt = '100' 
    AND a.stmon = \'{}\'  --년월 변경
    AND a.bukrs = '1000'
    AND a.cstat IN ('1','2')
'''.format(ym)) 
load_to_athena(conn, magam, 'default.monthly_deadline_{}'.format(ym))

<< Delete s3://adt-sandbox/_create/tables/default.monthly_deadline_202010
...tmp_db name = tmp_20201123_11_26_55_e70e28837f48899e9dd73fb699bce6c5a6d01f57
Start creating table
drop table if exists default.tmp_20201123_11_26_55_e70e28837f48899e9dd73fb699bce6c5a6d01f57
<< table not exist

        create  table         default.tmp_20201123_11_26_55_e70e28837f48899e9dd73fb699bce6c5a6d01f57
        with (format='Parquet', parquet_compression='GZIP')
        as
        
SELECT a.kcalss
	 , d.kunnr
	 , d.ctnum
     , a.conn_dept
     , a.conn_emp
     , a.cost_dept
     , a.cost_emp
     , a.ctnum as ctnum_hash
     , a.kunnr as kunnr_hash
     , a.cstat
     , a.gad_name
     , a.gad_start
     , a.conn_start
     , a.conn_end
     , CAST(a.monexp_amt AS DOUBLE)*100 AS monexp_amt
     , a.conn_type
     , a.ctype_l
     , a.ctype_m
     , a.ctype_s
     , a.intro_asp
     , a.itype_l
     , a.itype_m
     , a.itype_s
     , a.itype_sdesc
     , c.name1
     , a.gad_add01
     , a.post_add01
 

0

## 2. 기 가입고객 테이블

In [5]:
conn = athena_connection()
drop_table_in_athena(conn, 'default.tmupsell_exclude')
temp = load_from_athena(conn, '''
WITH t1 AS (
SELECT DISTINCT b.ctnum AS ctnum_hash
     , e.ctnum
     , b.rqnum
     , b.valid_sdate
     , b.valid_edate
     , b.ins_type
     , b.ins_amt
     , b.ins_amt_type
     , b.ins_payment
     , b.ins_stat
     , b.ins_user4
     , c.ins_stat_nm
     , c.ins_amt_type_nm
	 , f.gad_start
     , d.ctnum_old
  FROM (SELECT DISTINCT ctnum, ins_type, ins_amt, ins_amt_type, ins_payment, ins_stat, rqnum, valid_sdate, valid_edate, ins_user4
             FROM sap.zsdt4150
            WHERE ins_type IN ('02', '03', '05', '07')) b
LEFT JOIN (SELECT DISTINCT ctnum, rqnum, ins_stat_nm, ins_amt_type_nm FROM mart.zsdr6051) c
       ON b.ctnum = c.ctnum
      AND b.rqnum = c.rqnum
LEFT JOIN (SELECT DISTINCT ctnum, ctnum_old FROM mart.zsdr6001 WHERE ctnum_old <> '') d
       ON b.ctnum = d.ctnum
LEFT JOIN (SELECT ctnum, ctnum_hash FROM adt_work.hash_info) e
       ON b.ctnum = e.ctnum_hash
LEFT JOIN (SELECT ctnum, gad_start FROM sap.zsdt4013 WHERE stmon = \'{}\') f
       ON b.ctnum = f.ctnum
)

, t2 AS (
SELECT * , 'case1' AS info FROM t1 WHERE ins_stat = '1' AND ins_type = '02' AND ins_amt_type >= '20' UNION ALL
SELECT * , 'case2' AS info FROM t1 WHERE ins_stat = '1' AND ins_type = '03' AND ins_amt_type >= '50' UNION ALL
SELECT * , 'case3' AS info FROM t1 WHERE ins_stat = '1' AND ins_type = '07' UNION ALL
SELECT * , 'case4' AS info FROM t1 WHERE ins_stat = '1' AND ins_user4 = 'TM' UNION ALL
SELECT * , 'case5' AS info FROM t1 WHERE ins_stat = '1' AND valid_sdate >= \'{}\' AND (ctnum_old = '' OR (ctnum_old <> '' AND valid_sdate <> gad_start)) UNION ALL
SELECT * , 'case6' AS info FROM t1 WHERE ins_stat = '2' AND ins_type IN ('02', '03', '07') AND valid_edate >= \'{}\'
)

SELECT DISTINCT *
  FROM t2
 WHERE valid_sdate <> '99991231'
'''.format(ym, ym_minus_6month, ym_minus_6month))
load_to_athena(conn, temp, 'default.tmupsell_exclude'.format(ym))

<< Delete s3://adt-sandbox/_create/tables/default.tmupsell_exclude
...tmp_db name = tmp_20201123_11_29_31_0d2fde30266b08297809220a6e1325032905add8
Start creating table
drop table if exists default.tmp_20201123_11_29_31_0d2fde30266b08297809220a6e1325032905add8
<< table not exist

        create  table         default.tmp_20201123_11_29_31_0d2fde30266b08297809220a6e1325032905add8
        with (format='Parquet', parquet_compression='GZIP')
        as
        
WITH t1 AS (
SELECT DISTINCT b.ctnum AS ctnum_hash
     , e.ctnum
     , b.rqnum
     , b.valid_sdate
     , b.valid_edate
     , b.ins_type
     , b.ins_amt
     , b.ins_amt_type
     , b.ins_payment
     , b.ins_stat
     , b.ins_user4
     , c.ins_stat_nm
     , c.ins_amt_type_nm
	 , f.gad_start
     , d.ctnum_old
  FROM (SELECT DISTINCT ctnum, ins_type, ins_amt, ins_amt_type, ins_payment, ins_stat, rqnum, valid_sdate, valid_edate, ins_user4
             FROM sap.zsdt4150
            WHERE ins_type IN ('02', '03', '05', '07')) b
L

0

In [6]:
temp.to_csv('기가입고객.csv', encoding='utf-8-sig')

In [7]:
del temp

## 3. TM 실적 테이블 (계약번호, 회차, 최종접촉일, 최종접촉결과)

In [8]:
conn = athena_connection()
drop_table_in_athena(conn, 'default.tm_upsell_result_table')
tm = load_from_athena(conn, '''
WITH t1 AS (
SELECT DISTINCT a.custid, b.kunnr, CONCAT(dialdate, ' ; ', dialtime) AS dialdate, resultcd, teamcd
  FROM telepro.tm_list a
LEFT JOIN (adt_work.hash_info) b
      ON a.custid = b.ctnum
 WHERE teamcd IN ('313', '316')
)

, t2 AS(
SELECT a.kunnr, COUNT(a.resultcd) AS con_cnt, MAX(a.dialdate) AS final_dialdate
  FROM t1 a
LEFT JOIN adt_work.hash_info b
       ON a.kunnr = b.kunnr
GROUP BY a.kunnr
)

SELECT a.*, b.resultcd
  FROM t2 a
LEFT JOIN (SELECT * FROM t1) b
       ON a.kunnr = b.kunnr
      AND a.final_dialdate = b.dialdate
''')
load_to_athena(conn, tm, 'default.tm_upsell_result_table')

<< Delete s3://adt-sandbox/_create/tables/default.tm_upsell_result_table
...tmp_db name = tmp_20201123_11_30_31_85754159bdbd2cca272cf1d14e1dc6c1cdeab93f
Start creating table
drop table if exists default.tmp_20201123_11_30_31_85754159bdbd2cca272cf1d14e1dc6c1cdeab93f
<< table not exist

        create  table         default.tmp_20201123_11_30_31_85754159bdbd2cca272cf1d14e1dc6c1cdeab93f
        with (format='Parquet', parquet_compression='GZIP')
        as
        
WITH t1 AS (
SELECT DISTINCT a.custid, b.kunnr, CONCAT(dialdate, ' ; ', dialtime) AS dialdate, resultcd, teamcd
  FROM telepro.tm_list a
LEFT JOIN (adt_work.hash_info) b
      ON a.custid = b.ctnum
 WHERE teamcd IN ('313', '316')
)

, t2 AS(
SELECT a.kunnr, COUNT(a.resultcd) AS con_cnt, MAX(a.dialdate) AS final_dialdate
  FROM t1 a
LEFT JOIN adt_work.hash_info b
       ON a.kunnr = b.kunnr
GROUP BY a.kunnr
)

SELECT a.*, b.resultcd
  FROM t2 a
LEFT JOIN (SELECT * FROM t1) b
       ON a.kunnr = b.kunnr
      AND a.final_dialdate

0

In [9]:
df = load_from_athena(conn,'''
WITH magam AS (
SELECT a.*, b.con_cnt, b.final_dialdate, b.resultcd
  FROM default.monthly_deadline_{} a
LEFT JOIN default.tm_upsell_result_table b
       ON a.kunnr = b.kunnr
)

, t1 AS (
SELECT '1' AS filter, *  FROM magam WHERE cstat <> '1'
)

, t2 AS (
SELECT '2' AS filter, *  FROM magam WHERE ctype_l IN ('U00', 'U10', 'U20', 'U30', 'U40', 'U50', 'R00', 'W00')
                                     OR ctype_m IN ('W05', 'W06')
)

, t3 AS (
SELECT '3' AS filter, *  FROM magam WHERE conn_type NOT IN ('A01', 'A02')
)

, t4 AS (
SELECT '4' AS filter, *  FROM magam WHERE itype_l IN ('2', '3') OR itype_m = '22' OR itype_s IN ('160', '217', '222', '223')
)

, t5 AS (
SELECT '5' AS filter, *  FROM magam WHERE conn_type = 'A02' AND itype_m = '21'
)

, t6 AS (
SELECT '6' AS filter, *  FROM magam WHERE nsok IS NOT NULL
)

, t7 AS (
SELECT '7' AS filter, *  FROM magam WHERE gros_code > '0'
)

, t8 AS (
SELECT '8' AS filter, *  FROM magam WHERE gross_type = 'Y'
)

, t10 AS (
SELECT '10' AS filter, * FROM magam WHERE tm_yn <> 'Y'
)

, t11 AS (
SELECT '11' AS filter, * FROM magam WHERE ctnum IN (SELECT DISTINCT ctnum FROM default.tmupsell_exclude) -- 별도 테이블 작성 쿼리 참조 
)

, t13 AS (
SELECT '13' AS filter, * FROM magam WHERE (itype_l = '1' AND monexp_amt < 30000 OR monexp_amt > 400000)
)

, t14 AS (
SELECT '14' AS filter, * FROM magam WHERE first_bildate IS NULL OR first_bildate = '' OR first_bildate = ' ' OR SUBSTR(first_bildate, 1, 6) <> \'{}\'
)

, t15 AS (
SELECT '15' AS filter, * FROM magam WHERE kunnr_rownum > 1
)

, total AS (
SELECT * FROM t1 UNION ALL 
SELECT * FROM t2 UNION ALL 
SELECT * FROM t3 UNION ALL 
SELECT * FROM t4 UNION ALL 
SELECT * FROM t5 UNION ALL 
SELECT * FROM t6 UNION ALL 
SELECT * FROM t7 UNION ALL 
SELECT * FROM t8 UNION ALL
SELECT * FROM t10 UNION ALL
SELECT * FROM t11 UNION ALL
SELECT * FROM t13 UNION ALL
SELECT * FROM t14 UNION ALL
SELECT * FROM t15 
)

, final_filter AS (
SELECT ctnum, MIN(filter) AS filter FROM total GROUP BY ctnum
)

SELECT b.filter, a.* FROM magam a LEFT JOIN final_filter b ON a.ctnum = b.ctnum 
'''.format(ym, ym))

...tmp_db name = tmp_20201123_11_31_13_d59eaa216b11a2a4d122da07ba6ded108290dc65
Start creating table
drop table if exists default.tmp_20201123_11_31_13_d59eaa216b11a2a4d122da07ba6ded108290dc65
<< table not exist

        create  table         default.tmp_20201123_11_31_13_d59eaa216b11a2a4d122da07ba6ded108290dc65
        with (format='Parquet', parquet_compression='GZIP')
        as
        
WITH magam AS (
SELECT a.*, b.con_cnt, b.final_dialdate, b.resultcd
  FROM default.monthly_deadline_202010 a
LEFT JOIN default.tm_upsell_result_table b
       ON a.kunnr = b.kunnr
)

, t1 AS (
SELECT '1' AS filter, *  FROM magam WHERE cstat <> '1'
)

, t2 AS (
SELECT '2' AS filter, *  FROM magam WHERE ctype_l IN ('U00', 'U10', 'U20', 'U30', 'U40', 'U50', 'R00', 'W00')
                                     OR ctype_m IN ('W05', 'W06')
)

, t3 AS (
SELECT '3' AS filter, *  FROM magam WHERE conn_type NOT IN ('A01', 'A02')
)

, t4 AS (
SELECT '4' AS filter, *  FROM magam WHERE itype_l IN ('2', '3') OR it

In [10]:
df['filter'].value_counts(dropna=False)

14     494404
10     109338
13      92535
11      56012
NaN      6951
1        3881
2        1959
15       1138
5         363
4         199
7          89
8          12
3           1
Name: filter, dtype: int64

In [11]:
df.head()

,filter,kcalss,kunnr,ctnum,conn_dept,conn_emp,cost_dept,cost_emp,ctnum_hash,kunnr_hash,...,sk1_yn,sk2_yn,adt1_yn,aedat,aenam,nsok,kunnr_rownum,con_cnt,final_dialdate,resultcd
0,14,2,CU0328395,206108755,551800,K1940195,521700,K1110173,e76f76ef007f3fb67a04c965d2f4a0a1df71456f696223...,91914d4ab8e26095ea7737d1d2109d4df1769851468e9d...,...,None,None,None,None,None,None,1,1.0,2018/07/06 ; 15:00:59,SS03
1,13,N,CU0333690,220000788,521600,K1100442,521600,K1151305,fa45217d4ad49531d4f8255f24b0b00229f807540af7d0...,2ae5383b7c3d3e19617c7188655fea2ff493c02bef697b...,...,,,Y,20200106,BATCNTID,None,1,26.0,2017/09/21 ; 13:34:24,SE05
2,14,2,CU0351637,206141566,751200,K1990104,751200,K1030275,c1e3d5d975e59244c632489d7aeb690b4e626233df3c4e...,a17dbc6a640ad8f790cda58605cc1f54d3b6e4c80f8179...,...,None,None,None,None,None,None,1,5.0,2019/11/14 ; 09:54:28,SS03
3,14,N,CU0355724,219167407,511200,K1100105,511200,K1950079,bd386bfd02208d694326f5dda0665d6c4c03c71e9773cc...,35a1a6d3680976916e3dda60a9dd0918aca61c822e8638...,...,,,Y,20191022,BATCNTID,None,1,6.0,2020/07/30 ; 10:08:13,SE05
4,14,3,CU0355916,206147494,512000,K1030004,512000,K1181145,d98afb2bcb414f76ddd69c60b2442168e2fb07b204fd68...,9ce7aba8c5c5554682662771002c6090da0bfa2c48120d...,...,None,None,None,None,None,None,1,6.0,2019/05/14 ; 15:50:19,SS03


## 이 위가 15번 고객번호 중복 제거까지 반영한 내용. 이 아래부터 16번 ~ 23번 조건.

In [12]:
# 불필요한 컬럼 제거 
drop_col = ['kcalss', 'conn_dept', 'conn_emp', 'cost_dept', 'cost_emp', 'intro_asp', 'itype_sdesc', 'svc_str', 'pay_type', 'depo_type']
df.drop(drop_col, axis=1, inplace=True)

In [13]:
# 혹시 몰라서 카피떠놓기
df_copy = df.copy()

In [14]:
# 필터링하지 않을 이상전화번호 리스트 (이상한 전화번호지만 db에 남겨놓고 TM센터에서 수기로 다른 번호 찾기 시도)
no_delete = [
'00','01','02','03','010','0101','01111','010000','010226','0100000','0101111','0105256','0109152','0109364','01000000','01001638','02000000','010000000','010010010','010111111'
,'011111111','020000000','021111111','0100000000','0100000001','0100100000','0101010101','0101111111','0101111234','0101231234','0101234567','0110000000','0111110000','0111234567','0200000000'
,'0201021021','0202222222','0111111111','0211000000','0211111112','0212341234','0211112222','0222222222','0510000000','01000000000','01000000001','01000000250','01000000602','01000001111','01000009999'
,'01000010001','01000090000','01000112111','01001000000','01001000900','01001010101','01001111111','01010000000','01010001000','01011110000','01011111018','01011111110','01011111111'
,'01001001010','01011111112','01011111118','01011111234','01011111313','01011112222','01012341234','01012345678','01022222222','01100000000','01101111111','01101234567','01111111111'
,'01111112222','01111113333','01112311231','03200000000','05100000000','05100011111','05101111111','05101112222','010000000000','010111111111','011111111111','0100000000000','01000000000000'
,'010000000000000','0','1','2','3','10','101','1111','10000','10226','100000','101111','105256','109152','109364','1000000','1001638','2000000','10000000','10010010','10111111','11111111'
,'20000000','21111111','100000000','100000001','100100000','101010101','101111111','101111234','101231234','101234567','110000000','111110000','111234567','200000000','201021021','202222222'
,'111111111','211000000','211111112','212341234','211112222','222222222','510000000','1000000000','1000000001','1000000250','1000000602','1000001111','1000009999','1000010001','1000090000'
,'1000112111','1001000000','1001000900','1001010101','1001111111','1010000000','1010001000','1011110000','1011111018','1011111110','1011111111','1001001010','1011111112','1011111118','1011111234'
,'1011111313','1011112222','1012341234','1012345678','1022222222','1100000000','1101111111','1101234567','1111111111','1111112222','1111113333','1112311231','3200000000','5100000000','5100011111'
,'5101111111','5101112222','10000000000','10111111111','11111111111','100000000000','1000000000000','10000000000000']

In [15]:
# 이상휴대폰번호 판단 함수 

def valid_number(x):
      nums = list(str(x)) 
      n = len(nums)
      answer=[]
      i = 3
      
      while i <= n-3:
            if int(nums[i]) - int(nums[i+1]) == int(nums[i+1]) - int(nums[i+2]):
                  answer.append(True)
            else:
                  answer.append(False)
            i += 1
      
      if (all(answer) == True) or (str(x) in (no_delete)):
            return False
      else:
            return True     

In [23]:
# 16. 휴대폰번호 중복 제거

df['telf2'] = np.where(df['telf2'].str[0] != '0', "0"+df['telf2'], df['telf2'])
df['telf2'] = df['telf2'].str.replace('\D|\s', '', regex=True)

df['telf_valid_1'] = df['telf2'].str.contains('^01([0|1|6|7|8|9]?)-?([0-9]{3,4})-?([0-9]{4})$', regex=True)
df['telf_valid_2'] = df['telf2'].apply(valid_number)
df['telf_valid_final'] = np.where((df['telf_valid_1'] == True) & (df['telf_valid_2'] == True), True, False)
df['telf_valid_final'] = df['telf_valid_final'].map({True : 1, False : 0})

df = df.sort_values(by=['conn_type', 'ctnum'], axis=0, ascending=[True, False])                                   

df.loc[(df['filter'].isnull()==True) & (df[(df['filter'].isnull()==True) & ((df['telf_valid_final']==1))].duplicated(subset='telf2', keep='first'))
           , 'filter'] = 16

In [ ]:
# 17. 미수 내역

In [ ]:
# 18. 최근 1년 이내 민원접수 고객

In [ ]:
# 19. 해약 문의

In [ ]:
# 20. tm 거부

In [ ]:
# 21. 두낫콜

In [ ]:
# 22. 최종접촉일 3개월 이내

In [ ]:
# 23. 가입불가 필터링 (상호, 주소 등)